In [5]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import sys
import caffe
import os
# display plots in this notebook
%matplotlib inline
# set display defaults
plt.rcParams['figure.figsize'] = (10, 10)        # large images
plt.rcParams['image.interpolation'] = 'nearest'  # don't interpolate: show square pixels
plt.rcParams['image.cmap'] = 'gray'  # use grayscale output rather than a (potentially misleading) color heatmap

In [6]:
def load_list(data_dir, prefix = None, dataset = 1):
    train_list = os.path.join(data_dir, 'train') + str(dataset) + '.idx'
    test_list = os.path.join(data_dir, 'test') + str(dataset) + '.idx'
    with open(train_list) as f:
        train_list = f.read().splitlines()
    with open(test_list) as f:
        test_list = f.read().splitlines()
    if prefix is not None:
        train_list = map(lambda x: os.path.join(prefix, x), train_list)
        test_list = map(lambda x: os.path.join(prefix, x), test_list)
    return train_list, test_list

def load_img(filename, flip = False, grayscale = False):
    if grayscale:
        img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    else:
        img = cv2.imread(filename)
    if flip:
        img = cv2.flip(img, 1)
    if grayscale:
        width, height = img.shape
    else:
        width, height, _ = img.shape
    factor = max(256.0 / width, 256.0 / height)
    img = cv2.resize(img, None, fx = factor, fy = factor)
    return img

def load_imgs(filenames, reverse = False, flip = False, grayscale = False):
    if reverse:
        filenames = filenames[::-1]
    imgs = map(lambda x: load_img(x, flip, grayscale), filenames)
    return imgs

def load_video(video, data_dir, mask = None, reverse = False, flip = False, grayscale = False):
    filenames = os.listdir(os.path.join(data_dir, video))
    filenames = filter(lambda x: x[-4:] == 'jpeg', filenames)
    if mask is not None:
        filenames = filter(mask, filenames)
    filenames.sort()
    filenames = map(lambda x: os.path.join(data_dir, video, x), filenames)
    imgs = load_imgs(filenames, reverse, flip, grayscale)
    return imgs

def is_forward(video):
    if video[0] == 'F':
        return True
    return False

In [7]:
def load_flows(video):
    images = []
    labels = []
    for direction in ['f', 'b']:
        for flip in [False, True]:
            imgs_ = load_video(video, './data/ArrowDataAll', 
                               mask = lambda x: x[:3] == 'of' + direction, 
                               flip = flip, 
                               grayscale=True)
            imgs_ = map(lambda x: cv2.resize(x, (227, 227)), imgs_)
            imgs = []
            for i in range(len(imgs_)/2):
                imgs.append(np.asarray([imgs_[2*i], imgs_[2*i+1]]))
            images.extend(imgs)
            if direction == 'f':
                labels.extend([1]*len(imgs))
            else:
                labels.extend([0]*len(imgs))
    images = np.asarray(images)
    return images, np.asarray(labels)

In [8]:
def find_optimal_batch(n):
    n_opt = 4
    i = 4
    while i <= min(n, 20):
        if n % i == 0:
            n_opt = i
        i += 4
    return n_opt

def forward(videos, net):
    X = np.empty((0, 4096))
    y = np.empty(0)
    video_names = []
    for video in videos:
        print '[INFO] processing video %d / %d' % (videos.index(video) + 1, len(videos))
        images, labels = load_flows(video)
        images -= 128*np.ones_like(images)
        n_opt = find_optimal_batch(len(images))
        print '[INFO] %d images' % len(images)
        print '[INFO] processing batch of %d' % n_opt
        net.blobs['data'].reshape(n_opt, 2, 227, 227)
        for i in range(len(images)/n_opt):
            net.blobs['data'].data[...] = images[n_opt*i:n_opt*(i+1),...]
            output = net.forward()
            X = np.append(X, output['fc7'], axis = 0)
        y = np.append(y, labels)
        video_names.extend([video] * len(labels))
    return X, y, video_names

def forward_save(videos, net):
    for video in videos:
        print '[INFO] processing vtrain_list, test_list = load_list('./data', dataset = 1)ideo %d / %d' % (videos.index(video) + 1, len(videos))
        X = np.empty((0, 4096))
        y = np.empty(0)
        images, labels = load_flows(video)
        images -= 128*np.ones_like(images)
        n_opt = find_optimal_batch(len(images))
        print '[INFO] %d images' % len(images)
        print '[INFO] processing batch of %d' % n_opt
        net.blobs['data'].reshape(n_opt, 2, 227, 227)
        for i in range(len(images)/n_opt):
            net.blobs['data'].data[...] = images[n_opt*i:n_opt*(i+1),...]
            output = net.forward()
            X = np.append(X, output['fc7'], axis = 0)
        y = np.append(y, labels)
        with open(os.path.join('./data/ArrowDataAll', video, 'features.csv'), 'w') as f:
            np.savetxt(f, X, delimiter = ',', fmt = '%f')
        with open(os.path.join('./data/ArrowDataAll', video, 'labels.csv'), 'w') as f:
            np.savetxt(f, y, delimiter = ',', fmt = '%d')

In [9]:
def load_features(video):
    with open(os.path.join('./data/ArrowDataAll', video, 'features.csv'), 'r') as f:
        X = np.loadtxt(f, delimiter = ',')
    with open(os.path.join('./data/ArrowDataAll', video, 'labels.csv'), 'r') as f:
        y = np.loadtxt(f, delimiter = ',')
    video_names_with_multitude = [video] * len(y)
    return X, y, video_names_with_multitude

In [10]:
train_list, test_list = load_list('./data', dataset = 1)

In [11]:
X_train = np.empty((0, 4096))
y_train = np.empty(0)
video_train = []
for video in train_list[:60]:
    print train_list.index(video)+1
    X, y, video_names_with_multitude = load_features(video)
    X_train = np.append(X_train, X, axis = 0)
    y_train = np.append(y_train, y, axis = 0)
    video_train.extend(video_names_with_multitude)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60


In [12]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_jobs = -1)

In [13]:
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=-1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [14]:
X_test = np.empty((0, 4096))
y_test = np.empty(0)
video_test = []
for video in test_list[:30]:
    print test_list.index(video)+1
    X, y, video_names_with_multitude = load_features(video)
    X_test = np.append(X_test, X, axis = 0)
    y_test = np.append(y_test, y, axis = 0)
    video_test.extend(video_names_with_multitude)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30


In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 10)
pca.fit(X_train)
X_train_reduct = pca.transform(X_train)

In [17]:
rfc.score(X_train, y_train)

0.99032152230971127

In [16]:
X_train.shape

(48768, 4096)

In [18]:
X_test.shape

(24696, 4096)

In [ ]:
caffe.set_mode_gpu()
# load original model
model_def = 'models/bvlc_reference_caffenet/deploy.prototxt'
model_weights = 'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel'
net = caffe.Net(model_def, model_weights, caffe.TEST) 
# mean the conv1 layer across the 3 channels
conv1_mean = net.params['conv1'][0].data.mean(axis = 1)
conv1_stack = np.repeat(conv1_mean[:, np.newaxis, :, :], 2, axis = 1)

In [ ]:
# load modified model 
# with layers later than fc7 removed
model_def = 'models/caffenet2/deploy.prototxt'
model_weights = 'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel'
net = caffe.Net(model_def, model_weights, caffe.TEST) 
# network surgery
net.params['conv1-stack'][0].data[...] = conv1_stack

In [ ]:
svc = SVC(kernel = 'rbf')
svc.fit(X_train, y_train)

In [ ]:
svc.score(X_train, y_train)

In [ ]:
svc.predict(X_test)

In [ ]:
print 'raw precision %f' % svc.score(X_test, y_test)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 1000)
pca.fit(X_train)
X_train_reduct = pca.transform(X_train)
X_test_reduct = pca.transform(X_test)

In [ ]:
rfc = RandomForestClassifier(n_jobs=-1)
rfc.fit(X_train_reduct, y_train)
print rfc.score(X_train_reduct, y_train)
print rfc.score(X_test_reduct, y_test)

In [ ]:
rfc = RandomForestClassifier(n_jobs=-1)
rfc.fit(X_train, y_train)

In [ ]:
rfc.score(X_train, y_train)

In [ ]:
rfc.score(X_test, y_test)

In [ ]:
plt.plot(rfc.feature_importances_)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_jobs = -1)
rfc.fit(X_train, y_train)
rfc.score(X_test, y_test)

In [ ]:
X_train[:4]

In [ ]:
y_predict_ = svc.predict(X_test)
y_predict = np.empty(len(y_predict_) / 4)
y_test = np.empty(len(y_predict))
for i in range(len(y_predict)):
    k = 4*i
    y_predict[i] = y_predict_[k] + y_predict_[k+1] - y_predict_[k+2] - y_predict_[k+3]
    y_test[i] = y_test_[k]
y_predict = np.sign(y_predict + 0.5)
precision = np.sum(y_predict == y_test) / float(len(y_test))
print 'Precision of dataset %d: %f' % (dataset, precision)
return precisionsvc.score(X_test, y_test)